In [6]:
# https://www.freecodecamp.org/news/how-to-create-a-gui-quiz-application-using-tkinter-and-open-trivia-db/


import random
from tkinter import *
import requests

url = "https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
get_request = requests.get(url).json()

root = Tk()
root.title("Trivia")
root.geometry("1000x800")

canvas = Canvas(root, width=800, height=250, bg="lightblue2")
canvas.place(relx=.5, rely=.5,anchor= CENTER) 

game_display = canvas.create_text(200, 75, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

user_answer = StringVar()
user_answer.set("")

score = 0
questions_count=0  #overall amount of questions answered 
correct_question_count=0  #correct questions answered 

def generate_qlist(json):
    question_list = []
    for i in json:
        i = i["question"]   # i used professor gibson's code and built off it because i had a hard time getting everything to run in main 
        question_list.append(i)
    return question_list

def gen_question(list_):
    choice = random.choice(list_)
    return choice 
#   list_.remove(choice)  # here i was trying to figure out a way to remove the question from the list after using it  
                                            # so it wouldn't repeat but i couldn't figure it out. 
def generate_answers(json, question): 
    option_list = []
    for i in get_request:
        if i["question"] == question:
            cor = i["correctAnswer"]
            option_list.append(cor)
            for incor in i["incorrectAnswers"]: 
                option_list.append(incor)
    random.shuffle(option_list)
    return option_list

def ask_question(question, answers):
    canvas.itemconfig(game_display, text=question, width=300)
    
def gen_radiobuttons(answers):
    y_pos = 100
    for i in answers:
        rad_btn = Radiobutton(canvas, text=i, variable=user_answer, command=eval_score, value=i, font=('Helvetica 10 bold'))
        rad_btn.place(x=50, y=y_pos)
        y_pos+=20
    
def eval_score():
    answer = user_answer.get()
    for i in get_request:
        if i["question"] == current_question:
            if i["correctAnswer"] == answer:
                feedback["text"] = "Correct!"
                feedback.place(x=400, y=500)
                global score
                score += 10
                global questions_count #everytime question answered, counts to total questions answered
                questions_count += 1 
                global correct_question_count #everytime question answered correctly, added to correct question count
                correct_question_count += 1
            else:
                feedback["text"] = "incorrect :("
                feedback.place(x=400, y=500)
                questions_count += 1  #everytime question answered, counts to total questions answered


#def reset_all():   # was trying to come up with a way for the frame to reset so the previous choices didn't stay, but couldn't figure it out 
    #global canvas
    #canvas.destroy()

def score_display(): #created a function that displays the new score after every question is answered 
    score_display=Label(root, text=f'Score:{score}')
    feedback.place(x=500, y=500)
    score_display.pack()    

                
def main(): 
    global questions_count 
    if questions_count <= 10:  #limited the game to be 10 trivia questions, so it ended 
        q_list = generate_qlist(get_request)
        global current_question
        current_question = gen_question(q_list)
        ans_list = generate_answers(get_request, current_question)
        ask_question(current_question, ans_list)
        gen_radiobuttons(ans_list)
        score_display() #adding the function to display the score 
        #reset_all() # was trying to call function to reset frame, but couldn't figure it out 
    else: #displays feedback when 10 questions have been answered 
        feedback["text"]=f"The game has ended. You answered {correct_question_count}/10 questions correctly. Your overall score is {score}."
        feedback.place(x=200, y=500) #after 10 questions answered, gave users the total amount correct, and total points earned using f string 
        

new_question=Button(root,text="New Question", command=main)
new_question.pack() #created a button to make a new question appear 


main()
root.mainloop()